# Volumetric AE with the class weight

In [1]:
from utils import *

import os
import datetime

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

import sys
sys.path.append("/home/dongwonshin/Desktop/Open3D/build/lib")
from py3d import *

from VolumetricVAE import *

# Volumetric AE Model training

In [2]:
vol_ae = VolumetricAE(_kernel_size=3, _stride=1)
vol_ae = torch.nn.DataParallel(vol_ae, device_ids = range(torch.cuda.device_count()))
vol_ae.cuda()

optimizer = optim.Adam(vol_ae.parameters(), lr=0.0001)

class_weight = torch.Tensor(np.array(
    [0.025192871093750013,
     0.99568359375,
     0.9951199340820313,
     0.9965753173828125,
     0.998511962890625,
     0.9998382568359375,
     0.9990286254882812,
     0.9993521118164063,
     0.9994937133789062,
     0.9998776245117188,
     0.9997113037109375,
     1.0,
     0.9998117065429688,
     0.9989590454101562,
     0.9985369873046875,
     0.9993685913085938,
     1.0,
     1.0,
     0.9990896606445312,
     0.999835205078125,
     0.9960134887695312]
)).cuda()

recon_loss_func = torch.nn.CrossEntropyLoss(weight=class_weight)

In [6]:
epoches = 100
data_range = 100

for epoch in range(epoches):
    for idx in range(data_range):
        comp_pcd = read_point_cloud("/home/dongwonshin/Desktop/subvolume_dataset_making/mesh_results2/%06d_complete.ply" % idx)
        voxel_volume, color_volume = pcd2voxel(comp_pcd)
        
        input_volume = colorvol2classidvol(color_volume)
        input_volume = np.expand_dims(input_volume, axis=0) # fake batch axis
        input_volume = torch.from_numpy(input_volume).float() 
        input_volume = Variable(input_volume).cuda()

        optimizer.zero_grad()
        
        output_volume = vol_ae.forward(input_volume)
        
        target_volume = colorvol2classidvol(color_volume, False)
        target_volume = np.expand_dims(target_volume, axis=0) # fake batch axis
        target_volume = torch.from_numpy(target_volume).long() 
        target_volume = Variable(target_volume).cuda()
        
#         mean_sq = z_mean * z_mean
#         stddev_sq = z_sigma * z_sigma
#         kl_loss = 0.5 * torch.sum(mean_sq + stddev_sq - torch.log(stddev_sq) - 1)
        recon_loss = recon_loss_func(output_volume, target_volume)
        
        loss = recon_loss
        
        loss.backward()
        optimizer.step()

        l = loss.data[0]
    
    print(l)

    # save the intermediate result
    argmaxed = torch.argmax(output_volume,1)
    output_color_volume = classidvol2colorvol(argmaxed.cpu().numpy())
    viz_vvae_output(output_color_volume,'./v_ae_with_weight',show=False,save=True)
    

/home/dongwonshin/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1749: UserWarning: Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/home/dongwonshin/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:33: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(1.6171, device='cuda:0')
tensor(1.6093, device='cuda:0')
tensor(1.5799, device='cuda:0')
tensor(1.5664, device='cuda:0')
tensor(1.5528, device='cuda:0')
tensor(1.5315, device='cuda:0')
tensor(1.5185, device='cuda:0')
tensor(1.4880, device='cuda:0')
tensor(1.4161, device='cuda:0')
tensor(1.4273, device='cuda:0')
tensor(1.3743, device='cuda:0')
tensor(1.3149, device='cuda:0')
tensor(1.2883, device='cuda:0')
tensor(1.2743, device='cuda:0')
tensor(1.2519, device='cuda:0')
tensor(1.2382, device='cuda:0')
tensor(1.2337, device='cuda:0')
tensor(1.2265, device='cuda:0')
tensor(1.2224, device='cuda:0')
tensor(1.2091, device='cuda:0')
tensor(1.2067, device='cuda:0')
tensor(1.1934, device='cuda:0')
tensor(1.2194, device='cuda:0')
tensor(1.2018, device='cuda:0')
tensor(1.1749, device='cuda:0')
tensor(1.1739, device='cuda:0')
tensor(1.1955, device='cuda:0')
tensor(1.2116, device='cuda:0')
tensor(1.1674, device='cuda:0')
tensor(1.2100, device='cuda:0')
tensor(1.2240, device='cuda:0')
tensor(1